In [1]:
from agential.cog.critic.strategies.code import CriticCodeStrategy, CriticHEvalCodeStrategy
from agential.llm.llm import LLM
from dotenv import load_dotenv
load_dotenv()
import os
llm = LLM("gpt-3.5-turbo")


In [4]:
from agential.llm.llm import MockLLM


inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
tests = f"{inst['test']}\ncheck({inst['entry_point']})"

use_tool = True

responses = [
    '```python\ndef has_close_elements(numbers, threshold):\n    return any(abs(x - y) < threshold for i, x in enumerate(numbers) for j, y in enumerate(numbers) if i != j)\n```',
    "The function `has_close_elements` has a correct implementation, utilizing a generator expression with the `any` function to efficiently check if any two numbers in the list are closer to each other than the given threshold. The logic compares all pairs of numbers in the list except for pairs where the indices are the same, ensuring no number is compared with itself.\n\nThere are no issues with the function's design or implementation. The function correctly checks for close elements based on the specified threshold and passes the provided test cases successfully.\n\nTherefore, there are no problems with the given code.",
]
llm = MockLLM("gpt-3.5-turbo", responses=responses)
strat = CriticHEvalCodeStrategy(llm=llm, testing=True)
out = strat.generate(
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_HUMANEVAL,
    critique_examples=HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC if use_tool else HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_HUMANEVAL if use_tool else CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_HUMANEVAL,
    additional_keys={},
    critique_additional_keys={"tests": tests},
    max_interactions=3,
    use_tool=use_tool,
    reset=True
)

<PROMPT AGENT=================================================>
You are an AI that only responds with python code, NOT ENGLISH. You will be given a function signature and its docstring by the user.

```python
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

    pass
```
<PROMPT AGENT=================================================>
<OUT AGENT=================================================>
'```python\ndef has_close_elements(numbers, threshold):\n    return any(abs(x - y) < threshold for i, x in enumerate(numbers) for j, y in enumerate(numbers) if i != j)\n```'
<OUT AGENT=================================================>
<PROMPT CRITIC=================================================>
```p

In [5]:
out

CriticOutput(answer='def has_close_elements(numbers, threshold):\n    return any(abs(x - y) < threshold for i, x in enumerate(numbers) for j, y in enumerate(numbers) if i != j)', total_prompt_tokens=20, total_completion_tokens=40, total_tokens=60, total_prompt_cost=3e-05, total_completion_cost=7.999999999999999e-05, total_cost=0.00010999999999999999, total_prompt_time=1.0, total_time=0.5, additional_info=[CriticStepOutput(answer='def has_close_elements(numbers, threshold):\n    return any(abs(x - y) < threshold for i, x in enumerate(numbers) for j, y in enumerate(numbers) if i != j)', critique="The function `has_close_elements` has a correct implementation, utilizing a generator expression with the `any` function to efficiently check if any two numbers in the list are closer to each other than the given threshold. The logic compares all pairs of numbers in the list except for pairs where the indices are the same, ensuring no number is compared with itself.\n\nThere are no issues with t

In [2]:
from agential.cog.critic.prompts import CRITIC_CRITIQUE_INSTRUCTION_HUMANEVAL, CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_HUMANEVAL, CRITIC_POT_INSTRUCTION_HUMANEVAL, HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC, HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL
from agential.cog.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_POT


inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}
question = inst['prompt']
tests = f"{inst['test']}\ncheck({inst['entry_point']})"

use_tool = True

strat = CriticHEvalCodeStrategy(llm=llm)
out = strat.generate(
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_HUMANEVAL,
    critique_examples=HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC if use_tool else HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_HUMANEVAL if use_tool else CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_HUMANEVAL,
    additional_keys={},
    critique_additional_keys={"tests": tests},
    max_interactions=3,
    use_tool=use_tool,
    reset=True
)

<PROMPT AGENT=================================================>
You are an AI that only responds with python code, NOT ENGLISH. You will be given a function signature and its docstring by the user.

```python
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

    pass
```
<PROMPT AGENT=================================================>
<OUT AGENT=================================================>
'```python\ndef has_close_elements(numbers, threshold):\n    return any(abs(x - y) < threshold for i, x in enumerate(numbers) for j, y in enumerate(numbers) if i != j)\n```'
<OUT AGENT=================================================>
<PROMPT CRITIC=================================================>
```p

In [3]:
from agential.cog.critic.prompts import CRITIC_CRITIQUE_INSTRUCTION_MBPP, CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_MBPP, CRITIC_POT_INSTRUCTION_MBPP, MBPP_FEWSHOT_EXAMPLES_CRITIC, MBPP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL
from agential.cog.critic.strategies.code import CriticMBPPCodeStrategy
from agential.cog.fewshots.mbpp import MBPP_FEWSHOT_EXAMPLES_POT


question = "Write a python function to find the first repeated character in a given string."
tests = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""
use_tool = True

strat = CriticMBPPCodeStrategy(llm=llm)

out = strat.generate(
    question=question,
    examples=MBPP_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_MBPP,
    critique_examples=MBPP_FEWSHOT_EXAMPLES_CRITIC if use_tool else MBPP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_MBPP if use_tool else CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_MBPP,
    additional_keys={"tests": tests},
    critique_additional_keys={"tests": tests},
    max_interactions=3,
    use_tool=use_tool,
    reset=True
)

<PROMPT AGENT=================================================>
You are an expert Python programmer, and here is your task: Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

assert set(similar_elements((3, 4, 5, 6), (5, 7, 4, 10))) == set((4, 5))
assert set(similar_elements((1, 2, 3, 4), (5, 4, 3, 7))) == set((3, 4))
assert set(similar_elements((11, 12, 14, 13), (17, 15, 14, 13))) == set((13, 14))

```python
def similar_elements(test_tup1, test_tup2):
    res = tuple(set(test_tup1) & set(test_tup2))
    return res
```

---

You are an expert Python programmer, and here is your task: Write a python function to identify non-prime numbers.
Your code should pass these tests:

assert is_not_prime(2) == False
assert is_not_prime(10) == True
assert is_not_prime(35) == True
assert is_not_prime(37) == False

```python
import math

def is_not_prime(n):
    result = False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:


In [8]:
question = "Write a python function to find the first repeated character in a given string."
tests = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""
use_tool = True
responses = [
    'def first_repeated_char(s):\n    seen = set()\n    for char in s:\n        if char in seen:\n            return char\n        seen.add(char)\n    return None\n\n# Run the tests\nassert first_repeated_char("abcabc") == "a"\nassert first_repeated_char("abc") == None\nassert first_repeated_char("123123") == "1"',
    'There is no problem with the code provided. The function correctly finds the first repeated character in a given string by using a set to keep track of characters already seen. It returns the first character that appears more than once, or None if there are no repeated characters. The function passes the provided tests successfully.',
]
llm = MockLLM("gpt-3.5-turbo", responses=responses)
strat = CriticMBPPCodeStrategy(llm=llm, testing=True)

out = strat.generate(
    question=question,
    examples=MBPP_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_MBPP,
    critique_examples=MBPP_FEWSHOT_EXAMPLES_CRITIC if use_tool else MBPP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_MBPP if use_tool else CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_MBPP,
    additional_keys={"tests": tests},
    critique_additional_keys={"tests": tests},
    max_interactions=3,
    use_tool=use_tool,
    reset=True
)

<PROMPT AGENT=================================================>
You are an expert Python programmer, and here is your task: Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

assert set(similar_elements((3, 4, 5, 6), (5, 7, 4, 10))) == set((4, 5))
assert set(similar_elements((1, 2, 3, 4), (5, 4, 3, 7))) == set((3, 4))
assert set(similar_elements((11, 12, 14, 13), (17, 15, 14, 13))) == set((13, 14))

```python
def similar_elements(test_tup1, test_tup2):
    res = tuple(set(test_tup1) & set(test_tup2))
    return res
```

---

You are an expert Python programmer, and here is your task: Write a python function to identify non-prime numbers.
Your code should pass these tests:

assert is_not_prime(2) == False
assert is_not_prime(10) == True
assert is_not_prime(35) == True
assert is_not_prime(37) == False

```python
import math

def is_not_prime(n):
    result = False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:


In [9]:
out

CriticOutput(answer='def first_repeated_char(s):\n    seen = set()\n    for char in s:\n        if char in seen:\n            return char\n        seen.add(char)\n    return None\n\n# Run the tests\nassert first_repeated_char("abcabc") == "a"\nassert first_repeated_char("abc") == None\nassert first_repeated_char("123123") == "1"', total_prompt_tokens=20, total_completion_tokens=40, total_tokens=60, total_prompt_cost=3e-05, total_completion_cost=7.999999999999999e-05, total_cost=0.00010999999999999999, total_prompt_time=1.0, total_time=0.5, additional_info=[CriticStepOutput(answer='def first_repeated_char(s):\n    seen = set()\n    for char in s:\n        if char in seen:\n            return char\n        seen.add(char)\n    return None\n\n# Run the tests\nassert first_repeated_char("abcabc") == "a"\nassert first_repeated_char("abc") == None\nassert first_repeated_char("123123") == "1"', critique='There is no problem with the code provided. The function correctly finds the first repeate